# Convolution

La convolucion es una multiplicacion sofisticada.

$(f*g)(t)=\int_{-\infty}^{\infty}f(\tau)g(t-\tau)d\tau$


Imaginan se que estan trabajando en un hospital que trata a pacientes con una sola enfermedad. Para el plan de tratamiento cada paciente recibe 4 mili-litros de la cura. El primer dia llegan 2 paciente, el segundo dia 1 paciente, el tercer dia 5 pacientes, el cuarto dia y quinto dia 4 pacientes. Que cantidad de mediamentos utilizamos cada dia?

$[4]$ -> ml de medicamentos (plan)

$[2, 1, 5, 4, 4]$ -> numero de pacientes que llegan todos los dias.

1. dia: 2 paciente 4ml = neceistamos 8ml del medicamento -> $2 \times 4$
2. dia: 1 pacientes, 4ml = necesitamos 4ml del medicamento -> $1 \times 4$
3. dia: 5 pacientes, cada uno 4ml = necesitamos 20ml del medicamento -> $5 \times 4$
4. dia: 4 pacientes, cada uno 4ml = necesitamos 16ml del medicamento -> $4 \times 4$
5. dia: 4 pacientes, cada uno 4ml = necesitamos 16ml del medicamento -> $4 \times 4$

$[4] * [2,1,5,4,4] = [8,4,20,16,16]$

Si se multiplica el plan por la lista de pacientes, se obtiene la utilización para los próximos días.

Ahora la tratamos otra enfermadad mas complicado, para tratar la nueva enfermedad los pacientes deberian tomar 4ml el primer dia, despues 2ml el siguiente dia y 1ml el ultimo dia. Entonces:

Plan: $[4,2,1]$ -> ml de medicamentos

Asumimos la misma lista de pacientes $[2, 1, 5, 4, 4]$ y queremos saber cuantas ml necesitamos cada dia.

1. dia: 2 paciente 4ml = neceistamos 8ml del medicamento ($2 \times 4$)
2. dia: 1 pacientes, 4ml, mas 4ml para los 2 pacientes del dia anterior (2ml para cada uno) = necesitamos 4ml del medicamento -> $(1 \times 4) + (2 \times 2)$
3. dia: 5 pacientes, cada uno 4ml, mas 2ml del paciente del dia anterior mas 2ml de los dos pacientes del primer dia = necesitamos 24ml del medicamento -> $(5 \times 4) + (1 \times 2) + (2 \times 1)$
4. dia: 4 pacientes, cada uno 4ml, mas 10ml de los 5 pacientes del dia anterior mas 1ml del paciente del segundo dia = necesitamos 27ml del medicamento -> $(4 \times 4) + (5 \times 2) + (1 \times 1)$
5. dia: 4 pacientes, cada uno 4ml, mas 8ml de los 4 pacientes del dia anteroir mas 5ml de los 5 pacientes del tercer dia = necesitamos 29ml del medicamento -> $(4 \times 4) + (4 \times 2) + (5 \times 1)$


```
1. dia:

Plan:                             [4   2   1] 
Pacientes:        [4   4   5   1   2]

Medicamentos:                     [8]               = 8ml
```



```
2. dia:

Plan:                         [4   2   1] 
Pacientes:        [4   4   5   1   2]

Medicamentos:                 [4   4]               = 8ml
```



```
3. dia:

Plan:                      [4   2   1] 
Pacientes:        [ 4   4   5   1   2]

Medicamentos:              [20  2   2]               = 24ml
```



```
4. dia:

Plan:                 [4   2   1] 
Pacientes:        [4   4   5   1   2]
 
Medicamentos:         [16  10  1]                   = 27ml
```


```
5. dia:

Plan:             [4   2   1] 
Pacientes:        [4   4   5   1   2]

Medicamentos:     [16  8   5]                        = 29ml
```


```
6. dia:

Plan:         [4   2   1] 
Pacientes:        [4   4   5   1   2]

Medicamentos:     [8   4]                             = 12ml
```


```
7. dia:

Plan:     [4   2   1] 
Pacientes:        [4   4   5   1   2]

Medicamentos:     [4]                                = 4ml
```






In [ ]:
import numpy as np
import matplotlib.pylab as plt
from astropy.io import fits

In [ ]:
from matplotlib import rcParams

rcParams["figure.figsize"] = (15, 7)

In [ ]:
Pacientes = np.array( [2,1,5,4,4] )
Plan = np.array( [4,2,1] )


np.convolve(Pacientes, Plan)

## Imagenes astronomicos

Los imagenes obtenido a traves de un telescopio son una convolucion entre la imagen del objeto y la respuesta del instrumento.

![conv](markdown-img/Convolution_Illustrated_eng.png)


La luz del objeto llega desde el cielo y atravieza la atmosfera y todo un sistema optica complejo. El imagen que obtenemos un la CCD es una convolucion de un kernel que representa la respuesta del sistema con la imagen del objeto. La respuesta del sistema tambien se llama PSF (en ingles "Point Spread Function"), que hace la imagen mas difuso.


![telescope](markdown-img/27n.png)


In [ ]:
def unit_circle_vectorized(r):
    A = np.arange(-r,r+1)**2
    dists = np.sqrt(A[:,None] + A)
    return ((dists-r)<0.5).astype(int)

In [ ]:
unit_circle_vectorized(10).shape

In [ ]:
imagen_objeto = np.zeros( (100,100) )
imagen_objeto[40:61,40:61]=unit_circle_vectorized(10)
imagen_objeto = imagen_objeto *25

In [ ]:
plt.imshow(imagen_objeto)

In [ ]:
def crearGaussian(size, fwhm = 3, center=None):

    x = np.arange(0, size, 1, float)
    y = x[:,np.newaxis]

    if center is None:
        x0 = y0 = size // 2
    else:
        x0 = center[0]
        y0 = center[1]

    return np.exp(-np.log(2) * ((x-x0)**2 + (y-y0)**2) / fwhm**2)

In [ ]:
gaussian_psf = crearGaussian(10,fwhm=5)
plt.imshow(gaussian_psf)

In [ ]:
def convolution2d(image, kernel):
    m, n = kernel.shape
    if (m == n):
        y, x = image.shape
        y = y - m + 1
        x = x - m + 1
        new_image = np.zeros((y,x))
        for i in range(y):
            for j in range(x):
                new_image[i][j] = np.sum(image[i:i+m, j:j+m]*kernel) 
    return new_image

In [ ]:
imagen_ccd = convolution2d(imagen_objeto, gaussian_psf)
plt.imshow(imagen_ccd)

In [ ]:
f = plt.figure()
f.add_subplot(1,2,1)
plt.title('Imagen objeto')
plt.plot( imagen_objeto[42,:] )
plt.gca().axes.get_yaxis().set_visible(False)
f.add_subplot(1,2,2)
plt.title('Imagen CCD')
plt.plot( imagen_ccd[42,:] )
plt.gca().axes.get_yaxis().set_visible(False)